In [1]:
# Install required library
!pip install pydub


[notice] A new release of pip is available: 23.2.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [16]:
import os
from pydub import AudioSegment
from pathlib import Path

In [17]:
# Define paths
input_folder = "Supporting files/Century of Sounds select for player"
output_folder = "Supporting files/Century of Sounds select for player/mp3_output"

# Create output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

print(f"Input folder: {input_folder}")
print(f"Output folder: {output_folder}")

Input folder: Supporting files/Century of Sounds select for player
Output folder: Supporting files/Century of Sounds select for player/mp3_output


In [18]:
# Get all WAV files
wav_files = [f for f in os.listdir(input_folder) if f.lower().endswith('.wav')]
print(f"Found {len(wav_files)} WAV files:")
for wav_file in wav_files:
    print(f"  - {wav_file}")

Found 6 WAV files:
  - 008_remix.wav
  - 065_pitt.wav
  - 008_pitt.wav
  - 049_pitt.wav
  - 049_remix.wav
  - 065_remix.wav


In [19]:
# Convert each WAV file to MP3
for wav_file in wav_files:
    try:
        # Full paths
        wav_path = os.path.join(input_folder, wav_file)
        mp3_filename = os.path.splitext(wav_file)[0] + '.mp3'
        mp3_path = os.path.join(output_folder, mp3_filename)
        
        # Load WAV and export as MP3
        print(f"Converting {wav_file}...")
        audio = AudioSegment.from_wav(wav_path)
        # Set to 48kHz sample rate, 128kbps bitrate, stereo
        audio = audio.set_frame_rate(48000)
        audio = audio.set_channels(2)
        audio.export(mp3_path, format='mp3', bitrate='128k')
        
        # Add album art from matching JPG file
        # Extract first 3 digits from filename
        file_prefix = wav_file[:3]
        jpg_filename = f"{file_prefix}.jpg"
        jpg_path = os.path.join(input_folder, jpg_filename)
        
        if os.path.exists(jpg_path):
            # Add album art to MP3
            from mutagen.mp3 import MP3
            from mutagen.id3 import ID3, APIC
            
            audio_file = MP3(mp3_path, ID3=ID3)
            
            # Add ID3 tag if it doesn't exist
            try:
                audio_file.add_tags()
            except:
                pass
            
            # Read image file
            with open(jpg_path, 'rb') as img_file:
                audio_file.tags.add(
                    APIC(
                        encoding=3,  # UTF-8
                        mime='image/jpeg',
                        type=3,  # Cover (front)
                        desc='Cover',
                        data=img_file.read()
                    )
                )
            audio_file.save()
            print(f"  ✓ Saved to {mp3_filename} with album art from {jpg_filename}")
        else:
            print(f"  ✓ Saved to {mp3_filename} (no matching JPG found)")
        
    except Exception as e:
        print(f"  ✗ Error converting {wav_file}: {str(e)}")

print("\nConversion complete!")

Converting 008_remix.wav...
  ✓ Saved to 008_remix.mp3 with album art from 008.jpg
Converting 065_pitt.wav...
  ✓ Saved to 008_remix.mp3 with album art from 008.jpg
Converting 065_pitt.wav...
  ✓ Saved to 065_pitt.mp3 with album art from 065.jpg
Converting 008_pitt.wav...
  ✓ Saved to 065_pitt.mp3 with album art from 065.jpg
Converting 008_pitt.wav...
  ✓ Saved to 008_pitt.mp3 with album art from 008.jpg
Converting 049_pitt.wav...
  ✓ Saved to 008_pitt.mp3 with album art from 008.jpg
Converting 049_pitt.wav...
  ✓ Saved to 049_pitt.mp3 with album art from 049.jpg
Converting 049_remix.wav...
  ✓ Saved to 049_pitt.mp3 with album art from 049.jpg
Converting 049_remix.wav...
  ✓ Saved to 049_remix.mp3 with album art from 049.jpg
Converting 065_remix.wav...
  ✓ Saved to 049_remix.mp3 with album art from 049.jpg
Converting 065_remix.wav...
  ✓ Saved to 065_remix.mp3 with album art from 065.jpg

Conversion complete!
  ✓ Saved to 065_remix.mp3 with album art from 065.jpg

Conversion complete!


## Read MP3 Encoding Details

Read and display the encoding details of converted MP3 files.

In [6]:
# Install additional library for reading MP3 metadata
!pip install mutagen

  Obtaining dependency information for mutagen from https://files.pythonhosted.org/packages/b0/7a/620f945b96be1f6ee357d211d5bf74ab1b7fe72a9f1525aafbfe3aee6875/mutagen-1.47.0-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.4/194.4 kB 2.9 MB/s eta 0:00:00 0:00:01

[notice] A new release of pip is available: 23.2.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [20]:
from mutagen.mp3 import MP3
from mutagen.id3 import ID3

def get_mp3_details(mp3_path):
    """Read and return encoding details of an MP3 file"""
    try:
        audio = MP3(mp3_path)
        
        details = {
            'filename': os.path.basename(mp3_path),
            'duration': f"{audio.info.length:.2f} seconds",
            'bitrate': f"{audio.info.bitrate / 1000:.0f} kbps",
            'sample_rate': f"{audio.info.sample_rate} Hz",
            'channels': audio.info.channels,
            'mode': 'Stereo' if audio.info.channels == 2 else 'Mono',
            'file_size': f"{os.path.getsize(mp3_path) / 1024 / 1024:.2f} MB"
        }
        return details
    except Exception as e:
        return {'error': str(e)}

In [21]:
# Read encoding details for all MP3 files in the output folder
mp3_files = [f for f in os.listdir(output_folder) if f.lower().endswith('.mp3')]

print(f"Reading encoding details for {len(mp3_files)} MP3 files:\n")
print("=" * 80)

for mp3_file in mp3_files:
    mp3_path = os.path.join(output_folder, mp3_file)
    details = get_mp3_details(mp3_path)
    
    if 'error' in details:
        print(f"\n❌ {mp3_file}: Error - {details['error']}")
    else:
        print(f"\n📀 {details['filename']}")
        print(f"   Duration:    {details['duration']}")
        print(f"   Bitrate:     {details['bitrate']}")
        print(f"   Sample Rate: {details['sample_rate']}")
        print(f"   Channels:    {details['channels']} ({details['mode']})")
        print(f"   File Size:   {details['file_size']}")

print("\n" + "=" * 80)

Reading encoding details for 7 MP3 files:


📀 049_pitt.mp3
   Duration:    139.03 seconds
   Bitrate:     128 kbps
   Sample Rate: 48000 Hz
   Channels:    2 (Stereo)
   File Size:   3.79 MB

📀 049_remix.mp3
   Duration:    284.04 seconds
   Bitrate:     128 kbps
   Sample Rate: 48000 Hz
   Channels:    2 (Stereo)
   File Size:   6.01 MB

📀 065_remix.mp3
   Duration:    227.90 seconds
   Bitrate:     128 kbps
   Sample Rate: 48000 Hz
   Channels:    2 (Stereo)
   File Size:   4.44 MB

📀 8820738.mp3
   Duration:    272.14 seconds
   Bitrate:     128 kbps
   Sample Rate: 48000 Hz
   Channels:    2 (Stereo)
   File Size:   4.23 MB

📀 008_remix.mp3
   Duration:    178.03 seconds
   Bitrate:     128 kbps
   Sample Rate: 48000 Hz
   Channels:    2 (Stereo)
   File Size:   3.32 MB

📀 065_pitt.mp3
   Duration:    180.02 seconds
   Bitrate:     128 kbps
   Sample Rate: 48000 Hz
   Channels:    2 (Stereo)
   File Size:   3.71 MB

📀 008_pitt.mp3
   Duration:    155.35 seconds
   Bitrate:     128 

## Convert Excel Spreadsheet to JSON

Convert the Century select.xlsx spreadsheet to JSON format.

In [22]:
# Install required library for reading Excel files
!pip install pandas openpyxl


[notice] A new release of pip is available: 23.2.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [23]:
import pandas as pd
import json

In [24]:
# Read the Excel file
excel_path = "Supporting files/Century select.xlsx"
output_json_path = "Supporting files/Century_select.json"

# Read Excel file into DataFrame
df = pd.read_excel(excel_path)

print(f"Read {len(df)} rows from Excel file")
print(f"\nColumns: {list(df.columns)}")
print(f"\nFirst few rows:")
print(df.head())

Read 3 rows from Excel file

Columns: ['Number', 'City title', 'City artist', 'City description', 'Memory title', 'Memory artist', 'Memory description', 'Country', 'Cultural group', 'Year', 'Type ']

First few rows:
   Number           City title                                  City artist  \
0       8   Bayang men's songs                         Sonja Lois MITCHISON   
1      49  Chocó panpipe music  Jonathan AMBACHE and Richard SAUMAREZ SMITH   
2      65        Women singing                        Nicholas Justin ALLEN   

                                    City description         Memory title  \
0  From the sound collections of the Pitt Rivers ...          Bayang song   
1  From the sound collections of the Pitt Rivers ...         Little train   
2  From the sound collections of the Pitt Rivers ...  Bari women chanting   

         Memory artist                                 Memory description  \
0  Jaspal Singh Bhogal  This was one of the more difficult pieces I’ve...   
1   

In [25]:
# Convert DataFrame to JSON
# orient='records' creates a list of dictionaries (one per row)
json_data = df.to_dict(orient='records')

# Save to JSON file with pretty formatting
with open(output_json_path, 'w', encoding='utf-8') as f:
    json.dump(json_data, f, indent=2, ensure_ascii=False)

print(f"\n✓ Successfully converted to JSON: {output_json_path}")
print(f"Total records: {len(json_data)}")


✓ Successfully converted to JSON: Supporting files/Century_select.json
Total records: 3
